In [3]:
# Starting prices based on seashells

seashells = 1
seashells_to_sillicon_nuggets = 0.64
seashells_to_pizza = 1.98
seashells_to_snowballs = 1.34

# Starting prices based on sillicon nuggets
sillicon_nugets = 1
sillicon_nuggets_to_seashells = 1.49
sillicon_nugets_to_pizza = 3.1
sillicon_nugets_to_snowballs = 1.95

# Starting prices based on pizza
pizza = 1
pizza_to_seashells = 0.48
pizza_to_sillicon_nuggets = 0.31
pizza_to_snowballs = 0.7

# Starting prices based on snowballs
snowballs = 1
snowballs_to_seashells = 0.72
snowballs_to_sillicon_nuggets = 0.52
snowballs_to_pizza = 1.45

In [6]:
graph = {
    'seashells': {'sillicon_nugets': 0.64, 'pizza': 1.98, 'snowballs': 1.34},
    'sillicon_nugets': {'seashells': 1.49, 'pizza': 3.1, 'snowballs': 1.95},
    'pizza': {'seashells': 0.48, 'sillicon_nugets': 0.31, 'snowballs': 0.7},
    'snowballs': {'seashells': 0.72, 'sillicon_nugets': 0.52, 'pizza': 1.45}
    }


In [52]:
from itertools import product

start = 'seashells'
end = 'seashells'
best_path = []
initial_money = 500000
best_profit = 0

for path in product(graph.keys(), repeat=4):
    full_path = [start] + list(path) + [end]
    money = initial_money
    valid = True
    
    for i in range(len(full_path) - 1):
        src, dst = full_path[i], full_path[i+1]
        if dst in graph[src]:
            rate = graph[src][dst]
            money *= rate
        else:
            valid = False
            break
        
    if valid and money > best_profit:
        best_profit = money
        best_path = full_path
        
print(f'Path: {best_path}')
print(f'Money at the end: {best_profit}')
print(f'Profit: {best_profit - initial_money}')

Path: ['seashells', 'snowballs', 'sillicon_nuggets', 'pizza', 'snowballs', 'seashells']
Money at the end: 544340.16
Profit: 44340.16000000003


In [54]:
((best_profit - initial_money)/initial_money)*100

8.868032000000007

In [15]:
# Hardcoded calculation for the best path
initial_money * seashells_to_snowballs * snowballs_to_sillicon_nuggets * sillicon_nugets_to_pizza * pizza_to_snowballs * snowballs_to_seashells

544340.16

In [57]:
import math
import pandas as pd

# Graph with exchange rates
graph = {
    'seashells': {'sillicon_nuggets': 0.64, 'pizza': 1.98, 'snowballs': 1.34},
    'sillicon_nuggets': {'seashells': 1.49, 'pizza': 3.1, 'snowballs': 1.95},
    'pizza': {'seashells': 0.48, 'sillicon_nuggets': 0.31, 'snowballs': 0.7},
    'snowballs': {'seashells': 0.72, 'sillicon_nuggets': 0.52, 'pizza': 1.45}
}

currencies = list(graph.keys())
initial_money = 500_000
max_trades = 5
start = 'seashells'

# Build list of edges with weights = -log(rate)
edges = []
for src in graph:
    for dst in graph[src]:
        rate = graph[src][dst]
        weight = -math.log(rate)
        edges.append((src, dst, weight))

# Bellman-Ford: distance[currency][step] = best total weight to reach it in 'step' trades
distance = {cur: [float('inf')] * (max_trades + 1) for cur in currencies}
prev_node = {cur: [None] * (max_trades + 1) for cur in currencies}

distance[start][0] = 0  # Starting point: 0 cost at 0 trades

# Bellman-Ford over a limited number of steps (trades)
for k in range(1, max_trades + 1):
    for u, v, weight in edges:
        if distance[u][k - 1] + weight < distance[v][k]:
            distance[v][k] = distance[u][k - 1] + weight
            prev_node[v][k] = u

# Reconstruct the best profit path ending in 'seashells'
best_profit = -float('inf')
best_k = None

for k in range(1, max_trades + 1):
    if distance[start][k] < float('inf'):
        # Get profit by reversing the log
        profit = initial_money * math.exp(-distance[start][k])
        if profit > best_profit:
            best_profit = profit
            best_k = k

# Reconstruct path
path = []
cur = start
for i in range(best_k, 0, -1):
    path.append(cur)
    cur = prev_node[cur][i]
path.append(start)
path.reverse()

# Format result
result = {
    'Trades': best_k,
    'Path': ' -> '.join(path),
    'Final_Seashells': round(best_profit, 2),
    'Profit': round(best_profit - initial_money, 2)
}

# Show result in DataFrame format
df_result = pd.DataFrame([result])
print(df_result)


   Trades                                               Path  Final_Seashells  \
0       5  seashells -> snowballs -> sillicon_nuggets -> ...        544340.16   

     Profit  
0  44340.16  


In [62]:
path = df_result['Path']

print(path.iloc[0])

seashells -> snowballs -> sillicon_nuggets -> pizza -> snowballs -> seashells
